<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/evaluation/probe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 5.5 MB 27.9 MB/s 
     |████████████████████████████████| 182 kB 17.3 MB/s 
     |████████████████████████████████| 7.6 MB 50.4 MB/s 


In [ ]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 24.8 MB/s 


In [ ]:
import os
import numpy as np
import pandas as pd

import torch

from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


### Black

In [ ]:
!pip install black[jupyter]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 14.0 MB/s 
     |████████████████████████████████| 843 kB 56.0 MB/s 
     |████████████████████████████████| 96 kB 6.1 MB/s 
     |████████████████████████████████| 1.6 MB 49.0 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2


In [ ]:
!black "/content/drive/MyDrive/Colab Notebooks/probe.ipynb"

### Probe Class

A probe instance can be used to generate predictions from any of our models.

It loads (and caches) appropriate tokenizers and models as needed.

The only method you need to use is the predict method.

In [ ]:
class Probe:
    """
    A class used to represent an Awesome Puppies Question Generation Probe
    ...

    Attributes
    ----------
    models : dict
        a dictionary of cached models
    tokenizers : dict
        a dictionary of cached tokenizers

    Methods
    -------
    predict(context, answer, base_model='bart', training_dataset='amalgam',
            num_beams=1, early_stopping=False, no_repeat_ngram_size=0,
            maximum_input_length=1024, maximum_target_length=50)

        returns a prediction string

    retrieve_tokenizer (base_model='bart')

        returns and caches a tokenizer

    retrieve_model(base_model='bart', training_dataset='amalgam')

        returns and caches a model
    """

    project_root = "/content/drive/MyDrive/w266 NLP Final Project/"
    model_root = project_root + "Models/"

    def __init__(self):
        self.models = {}
        self.tokenizers = {}

    def predict(
        self,
        context,
        answer,
        base_model="bart",
        training_dataset="amalgam",
        num_beams=1,
        early_stopping=False,
        no_repeat_ngram_size=0,
        maximum_input_length=1024,
        maximum_target_length=50,
    ):

        tokenizer = self.retrieve_tokenizer(base_model)
        model = self.retrieve_model(base_model, training_dataset)

        if base_model == "bart":
            prompt_string = f"{answer} </s> {context}"
        elif base_model == "T5":
            prompt_string = f"generate question: answer: {answer} context: {context}"
        else:
            raise ValueError("invalid base model")

        inputs = tokenizer(
            prompt_string,
            return_tensors="pt",
            max_length=maximum_input_length,
            truncation=True,
            padding=True,
        )
        output_ids = model.generate(
            inputs["input_ids"].cuda(),
            max_length=maximum_target_length,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=early_stopping,
        )
        prediction = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        return prediction

    def retrieve_tokenizer(self, base_model="bart"):
        tokenizer = self.tokenizers.get(base_model)
        if tokenizer:
            return tokenizer

        if base_model == "bart":
            tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
        elif base_model == "T5":
            tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")
        else:
            raise ValueError("invalid base model")

        self.tokenizers[base_model] = tokenizer
        return tokenizer

    def retrieve_model(self, base_model="bart", training_dataset="amalgam"):
        model_tuple = (base_model, training_dataset)
        model = self.models.get(model_tuple)
        if model:
            return model

        model_dir = f"{self.model_root}{base_model}_base_pt_long.{training_dataset}"

        if base_model == "bart":
            print(f"Loading: {model_dir}")
            model = BartForConditionalGeneration.from_pretrained(model_dir)
        elif base_model == "T5":
            print(f"Loading: {model_dir}")
            model = T5ForConditionalGeneration.from_pretrained(model_dir)
        else:
            raise ValueError("invalid base model")

        model.to(torch.device("cuda:0"))
        self.models[model_tuple] = model
        return model

### Play Time

In [ ]:
help(Probe)

Help on class Probe in module __main__:

class Probe(builtins.object)
 |  A class used to represent an Awesome Puppies Question Generation Probe
 |  ...
 |  
 |  Attributes
 |  ----------
 |  models : dict
 |      a dictionary of cached models
 |  tokenizers : dict
 |      a dictionary of cached tokenizers
 |  
 |  Methods
 |  -------
 |  predict(context, answer, base_model='bart', training_dataset='amalgam',
 |          num_beams=1, early_stopping=False, no_repeat_ngram_size=0,
 |          maximum_input_length=1024, maximum_target_length=50)
 |  
 |      returns a prediction string
 |  
 |  retrieve_tokenizer (base_model='bart')
 |  
 |      returns and caches a tokenizer
 |  
 |  retrieve_model(base_model='bart', training_dataset='amalgam')
 |  
 |      returns and caches a model
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  predict(self, context, answer, base_model='bart', training_dataset='amal

In [ ]:
p = Probe()

In [ ]:
context = "Mary had a little lamb, its fleece was white as snow. Everywhere the child went, the little lamb was sure to go."
answer = "Mary"

In [ ]:
p.predict(context, answer)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Loading: /content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_long.amalgam


['who had a little lamb in her hair']

In [ ]:
p.predict(context, answer, base_model="T5")

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/605 [00:00<?, ?B/s]

Loading: /content/drive/MyDrive/w266 NLP Final Project/Models/T5_base_pt_long.amalgam


['who had a little lamb in the bible']

In [ ]:
p.predict(context, answer, maximum_target_length=8)

['who had a little lamb']

In [ ]:
p.predict(context, answer, maximum_target_length=7, base_model="T5")

['who had a little lamb']

In [ ]:
p.predict(
    context,
    answer,
    maximum_target_length=8,
    base_model="bart",
    training_dataset="squad",
)

Loading: /content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_long.squad


['Who had a little lamb']

In [ ]:
list(p.models.keys())

[('bart', 'amalgam'), ('T5', 'amalgam'), ('bart', 'squad')]